In [1]:
import pandas as pd
import seaborn as sns
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters

# Grade Analysis

In [2]:
grade_df = pd.read_csv('Menagerie/data/grades.csv', index_col=0)
grade_df

,assignment_number,comments,skill,participant_id,batch,grade
0,18.0,The code use meaningful identifier names and a...,Readability,15,1,B+
1,620.0,Code is well organized into logical classes an...,Readability,15,1,A+
2,466.0,"The code is well-organized, consistently inden...",Readability,15,1,A-
3,591.0,The code is well organized and readable. Inden...,Readability,15,1,A-
4,587.0,The code is well organized and readable. Inden...,Readability,15,1,A-
...,...,...,...,...,...,...
15,58.0,NaN,Documentation,14,1,A++
16,353.0,NaN,Documentation,14,1,A++
17,422.0,NaN,Documentation,14,1,A++
18,480.0,NaN,Documentation,14,1,A++


In [9]:
readability_df = grade_df[grade_df['skill'] == 'Readability'][['participant_id', 'assignment_number', 'grade']]
readability_df

,participant_id,assignment_number,grade
0,15,18.0,B+
1,15,620.0,A+
2,15,466.0,A-
3,15,591.0,A-
4,15,587.0,A-
...,...,...,...
15,14,58.0,A++
16,14,353.0,A++
17,14,422.0,A++
18,14,480.0,A++


In [45]:
readability_batch_df = readability_df[readability_df['participant_id'].isin([1, 2, 3, 4])].sort_values(by='participant_id')
readability_batch_df

,participant_id,assignment_number,grade
19,1,144.0,B
18,1,327.0,A
17,1,219.0,A
16,1,481.0,B
15,1,218.0,A
...,...,...,...
13,4,453.0,A
12,4,143.0,A-
11,4,223.0,A-
15,4,234.0,A+


In [49]:
pivot = readability_batch_df.pivot(index='assignment_number', columns='participant_id', values='grade')
pivot

participant_id,1,2,3,4
assignment_number,,,,
29.0,F,F,F,F
36.0,C,A+,A++,B
59.0,A+,B+,A++,B+
111.0,B,A-,A++,B
119.0,B,B-,A++,B+
143.0,B,B,A++,A-
144.0,B,A-,A++,A++
188.0,A-,B,A++,A
218.0,A,A,A++,A-


In [50]:
agg_raters = aggregate_raters(pivot)
agg_raters

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
        [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
        [1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2],
        [1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0],
        [1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
        [2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
        [2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [0,

In [51]:
fleiss_kappa(agg_raters[0], 'fleiss')

-0.0028740903809698452